## Merging hourly taxi, weather, and incoming passenger data for 2017

### Count of Taxi Pickups per hour

In [2]:
import numpy as np
from tqdm import tqdm
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [2]:
def taxi_resample(path):
    taxi_df = pd.read_csv(path, compression='gzip')
    taxi_df["tpep_pickup_datetime"] = pd.DatetimeIndex(taxi_df.tpep_pickup_datetime).tz_localize('America/New_York')
    taxi_df["tpep_dropoff_datetime"] = pd.DatetimeIndex(taxi_df.tpep_dropoff_datetime).tz_localize('America/New_York')
    taxi_df.set_index("tpep_pickup_datetime", inplace = True)
    taxi_hourly = taxi_df.resample('H')['passenger_count'].count()
    taxi_hourly_df = taxi_hourly.to_frame()
    taxi_hourly_df.rename(columns={'passenger_count':'pickup_count'}, inplace = True)
    return taxi_hourly_df

In [3]:
taxi_hourly_df = taxi_resample('../clean_data/TaxiData_Jan17-Jun17.gz')

### Weather Data

In [3]:
check = pd.read_csv("../clean_data/weather_NY.csv")

In [4]:
check.head()

,dt,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,pressure,...,rain_today,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1349096400,2012-10-01 13:00:00 +0000 UTC,5128581,NaN,NaN,NaN,288.22,286.15,294.26,1012,...,NaN,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02d
1,1349186400,2012-10-02 14:00:00 +0000 UTC,5128581,NaN,NaN,NaN,290.15,288.15,294.82,1012,...,NaN,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02d
2,1349190000,2012-10-02 15:00:00 +0000 UTC,5128581,NaN,NaN,NaN,291.32,289.82,296.48,1012,...,NaN,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02d
3,1349193600,2012-10-02 16:00:00 +0000 UTC,5128581,NaN,NaN,NaN,291.78,290.15,295.93,0,...,0.0,NaN,NaN,NaN,NaN,92,800,Clear,sky is clear,02d
4,1349197200,2012-10-02 17:00:00 +0000 UTC,5128581,NaN,NaN,NaN,292.74,291.15,296.48,0,...,0.0,NaN,NaN,NaN,NaN,92,800,Clear,sky is clear,02d


In [8]:
check["date"] = pd.DatetimeIndex(pd.to_datetime(check.dt, unit="s")) \
                    .tz_localize('UTC').tz_convert('America/New_York')

In [10]:
check.set_index("date", inplace=True)

In [19]:
check

,dt,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,pressure,...,rain_today,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
date,,,,,,,,,,,,,,,,,,,,,
2012-10-01 09:00:00-04:00,1349096400,2012-10-01 13:00:00 +0000 UTC,5128581,NaN,NaN,NaN,288.22,286.15,294.26,1012,...,NaN,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02d
2012-10-02 10:00:00-04:00,1349186400,2012-10-02 14:00:00 +0000 UTC,5128581,NaN,NaN,NaN,290.15,288.15,294.82,1012,...,NaN,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02d
2012-10-02 11:00:00-04:00,1349190000,2012-10-02 15:00:00 +0000 UTC,5128581,NaN,NaN,NaN,291.32,289.82,296.48,1012,...,NaN,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02d
2012-10-02 12:00:00-04:00,1349193600,2012-10-02 16:00:00 +0000 UTC,5128581,NaN,NaN,NaN,291.78,290.15,295.93,0,...,0.0,NaN,NaN,NaN,NaN,92,800,Clear,sky is clear,02d
2012-10-02 13:00:00-04:00,1349197200,2012-10-02 17:00:00 +0000 UTC,5128581,NaN,NaN,NaN,292.74,291.15,296.48,0,...,0.0,NaN,NaN,NaN,NaN,92,800,Clear,sky is clear,02d
2012-10-02 14:00:00-04:00,1349200800,2012-10-02 18:00:00 +0000 UTC,5128581,NaN,NaN,NaN,293.82,293.15,295.15,1011,...,NaN,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03d
2012-10-02 15:00:00-04:00,1349204400,2012-10-02 19:00:00 +0000 UTC,5128581,NaN,NaN,NaN,294.08,293.15,295.15,0,...,0.0,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,02d
2012-10-02 16:00:00-04:00,1349208000,2012-10-02 20:00:00 +0000 UTC,5128581,NaN,NaN,NaN,294.72,294.15,295.37,1012,...,NaN,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03d
2012-10-02 17:00:00-04:00,1349211600,2012-10-02 21:00:00 +0000 UTC,5128581,NaN,NaN,NaN,294.85,293.15,295.93,0,...,0.0,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,02d


In [4]:
def clean_weather(path):
    weather_df = pd.read_csv(path)
    weather_df["date"] = pd.DatetimeIndex(pd.to_datetime(weather_df.dt, unit="s")) \
                    .tz_localize('UTC').tz_convert('America/New_York')
    weather_df.drop_duplicates(['date'], keep='first', inplace = True)
    weather_df.set_index("date", inplace=True)
    
    weather_df.drop(["dt", "dt_iso", "city_id", "city_name", "lat", "lon", "weather_icon", "grnd_level", \
                 "sea_level", "clouds_all", "weather_id", "rain_1h", "rain_3h", "rain_24h", \
                 "rain_today", "snow_1h", "snow_3h", "snow_24h", "snow_today"], axis=1, inplace=True)
    return weather_df

In [5]:
weather_df = clean_weather("../clean_data/weather_NY.csv")

### Merge Taxi and Weather Data

In [6]:
taxi_weather = taxi_hourly_df.merge(right=weather_df, how='left', left_index=True, right_index=True)
taxi_weather.index = taxi_weather.index.tz_convert('America/New_York')

In [7]:
taxi_weather_filled = taxi_weather.fillna(method='ffill')

In [8]:
taxi_weather_filled.head()

,pickup_count,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,weather_main,weather_description
tpep_pickup_datetime,,,,,,,,,,
2017-01-01 00:00:00-05:00,53,279.07,277.15,280.15,1013.0,45.0,2.0,250.0,Clouds,overcast clouds
2017-01-01 01:00:00-05:00,3,279.14,277.15,281.15,1013.0,45.0,2.0,260.0,Clouds,overcast clouds
2017-01-01 02:00:00-05:00,0,279.21,276.15,283.15,1012.0,45.0,3.0,270.0,Rain,light rain
2017-01-01 03:00:00-05:00,4,279.43,277.15,285.15,1013.0,45.0,3.0,270.0,Clouds,overcast clouds
2017-01-01 04:00:00-05:00,4,279.50,276.15,286.15,1014.0,42.0,4.0,280.0,Clouds,scattered clouds


### Passenger seat data

In [9]:
def clean_seats(path):
    seats_per_hour = pd.read_csv(path, header=None)
#     print(seats_per_hour.head())
    seats_per_hour.columns = ['Time', 'Passengers']
    seats_per_hour["Time"] = pd.DatetimeIndex(seats_per_hour.Time)
    seats_per_hour.set_index("Time", inplace = True)

    #To get rid of errors surrounding daylight savings time. All of taxi pickups and passenger counts are NaNs for these times
    seats_per_hour = seats_per_hour.drop([pd.Timestamp('2016-03-13 02:00:00'), pd.Timestamp('2016-11-06 01:00:00'), pd.Timestamp('2017-03-12 02:00:00')])
    seats_per_hour.index = seats_per_hour.index.tz_localize('America/New_York')
    return seats_per_hour

In [10]:
seats_per_hour = clean_seats("../clean_data/seats_per_hour.csv")

### Merging all dataframes

In [11]:
merged_df = pd.merge(left=taxi_weather_filled, right=seats_per_hour, how='inner', left_index=True, right_index=True)

In [12]:
merged_df.head(20)

,pickup_count,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,weather_main,weather_description,Passengers
2017-01-01 00:00:00-05:00,53,279.07,277.15,280.15,1013.0,45.0,2.0,250.0,Clouds,overcast clouds,NaN
2017-01-01 01:00:00-05:00,3,279.14,277.15,281.15,1013.0,45.0,2.0,260.0,Clouds,overcast clouds,150.0
2017-01-01 02:00:00-05:00,0,279.21,276.15,283.15,1012.0,45.0,3.0,270.0,Rain,light rain,NaN
2017-01-01 03:00:00-05:00,4,279.43,277.15,285.15,1013.0,45.0,3.0,270.0,Clouds,overcast clouds,NaN
2017-01-01 04:00:00-05:00,4,279.50,276.15,286.15,1014.0,42.0,4.0,280.0,Clouds,scattered clouds,NaN
2017-01-01 05:00:00-05:00,6,279.21,276.15,286.15,1015.0,48.0,2.0,280.0,Clouds,overcast clouds,NaN
2017-01-01 06:00:00-05:00,4,278.80,276.15,286.15,1016.0,52.0,2.0,290.0,Clear,sky is clear,NaN
2017-01-01 07:00:00-05:00,35,277.84,275.15,281.15,1017.0,55.0,2.0,220.0,Clouds,scattered clouds,272.0
2017-01-01 08:00:00-05:00,101,277.30,274.15,281.15,1018.0,64.0,2.0,220.0,Clear,sky is clear,1130.0
2017-01-01 09:00:00-05:00,147,277.98,276.15,281.15,1020.0,48.0,2.0,220.0,Clouds,scattered clouds,564.0


In [13]:
merged_df.to_csv("../clean_data/2017_combined_data.csv")